In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g`. pd.read_csv)
from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from collections import Counter
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier

# Data Ingestion

In [2]:
data ='hypothyroid.csv'

In [3]:
df = pd.read_csv(data)

In [4]:
# see all columns and records
pd.set_option('display.max_columns', None)
df.head(10)

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,query_hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH_measured,TSH,T3_measured,T3,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Class
0,41,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,1.3,t,2.5,t,125,t,1.14,t,109,f,?,SVHC,negative
1,23,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,4.1,t,2,t,102,f,?,f,?,f,?,other,negative
2,46,M,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.98,f,?,t,109,t,0.91,t,120,f,?,other,negative
3,70,F,t,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.16,t,1.9,t,175,f,?,f,?,f,?,other,negative
4,70,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.72,t,1.2,t,61,t,0.87,t,70,f,?,SVI,negative
5,18,F,t,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.03,f,?,t,183,t,1.3,t,141,f,?,other,negative
6,59,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,?,f,?,t,72,t,0.92,t,78,f,?,other,negative
7,80,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,2.2,t,0.6,t,80,t,0.7,t,115,f,?,SVI,negative
8,66,F,f,f,f,f,f,f,f,f,f,f,f,t,f,f,t,0.6,t,2.2,t,123,t,0.93,t,132,f,?,SVI,negative
9,68,M,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,2.4,t,1.6,t,83,t,0.89,t,93,f,?,SVI,negative


In [5]:
# replace ? with nan
df.replace('?', np.nan, inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   age                        3771 non-null   object 
 1   sex                        3622 non-null   object 
 2   on_thyroxine               3772 non-null   object 
 3   query_on_thyroxine         3772 non-null   object 
 4   on_antithyroid_medication  3772 non-null   object 
 5   sick                       3772 non-null   object 
 6   pregnant                   3772 non-null   object 
 7   thyroid_surgery            3772 non-null   object 
 8   I131_treatment             3772 non-null   object 
 9   query_hypothyroid          3772 non-null   object 
 10  query_hyperthyroid         3772 non-null   object 
 11  lithium                    3772 non-null   object 
 12  goitre                     3772 non-null   object 
 13  tumor                      3772 non-null   objec

In [7]:
# check for missing values greater than 0
missing = df.isnull().sum()
missing = missing[missing > 0]

print(f"Missing values: \n{missing}")
print(f"Total records in dataset: {len(df)}")

Missing values: 
age       1
sex     150
TSH     369
T3      769
TT4     231
T4U     387
FTI     385
TBG    3772
dtype: int64
Total records in dataset: 3772


##### Observations

- from above we can see that there are 3772 records and 30 columns
- we can see age, TSH, T3, TT4, T4U, FTI are numerical columns, but they are object type so we need to convert them to numerical type
- we can see that there are missing values in some columns
- column TBG we can drop 

## Feature Engineering

In [8]:
df.drop('TBG',inplace=True,axis=1)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   age                        3771 non-null   object
 1   sex                        3622 non-null   object
 2   on_thyroxine               3772 non-null   object
 3   query_on_thyroxine         3772 non-null   object
 4   on_antithyroid_medication  3772 non-null   object
 5   sick                       3772 non-null   object
 6   pregnant                   3772 non-null   object
 7   thyroid_surgery            3772 non-null   object
 8   I131_treatment             3772 non-null   object
 9   query_hypothyroid          3772 non-null   object
 10  query_hyperthyroid         3772 non-null   object
 11  lithium                    3772 non-null   object
 12  goitre                     3772 non-null   object
 13  tumor                      3772 non-null   object
 14  hypopitu

In [10]:
percent_null = 100 * df.isnull().sum() / len(df)
percent_null = percent_null[percent_null > 0].sort_values()
data_types = df.dtypes

# Combine into a single DataFrame for display
nulls_with_dtypes = pd.DataFrame({
    'percent_null': percent_null,
    'data_type': data_types[percent_null.index]
})

print(nulls_with_dtypes)

     percent_null data_type
age      0.026511    object
sex      3.976670    object
TT4      6.124072    object
TSH      9.782609    object
FTI     10.206787    object
T4U     10.259809    object
T3      20.387063    object


In [11]:
# Drop specified columns with high percentage of missing values
columns_to_drop = ['TT4', 'TSH', 'FTI', 'T4U', 'T3']
df = df.drop(columns=columns_to_drop)

# Drop rows where 'age' or 'sex' is null
df = df.dropna(subset=['age', 'sex'])


In [12]:
# check for missing values greater than 0
missing = df.isnull().sum()
missing = missing[missing > 0]

print(f"Missing values: \n{missing}")
print(f"Total records in dataset: {len(df)}")

Missing values: 
Series([], dtype: int64)
Total records in dataset: 3621


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3621 entries, 0 to 3771
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   age                        3621 non-null   object
 1   sex                        3621 non-null   object
 2   on_thyroxine               3621 non-null   object
 3   query_on_thyroxine         3621 non-null   object
 4   on_antithyroid_medication  3621 non-null   object
 5   sick                       3621 non-null   object
 6   pregnant                   3621 non-null   object
 7   thyroid_surgery            3621 non-null   object
 8   I131_treatment             3621 non-null   object
 9   query_hypothyroid          3621 non-null   object
 10  query_hyperthyroid         3621 non-null   object
 11  lithium                    3621 non-null   object
 12  goitre                     3621 non-null   object
 13  tumor                      3621 non-null   object
 14  hypopituitary

In [14]:
# befor filling missing values, we need to fix the data types of the columns

# Convert 'age' to integer
df['age'] = df['age'].astype(int)


In [15]:
# print the unique values in categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    print(f"{col}: {df[col].unique()}")

sex: ['F' 'M']
on_thyroxine: ['f' 't']
query_on_thyroxine: ['f' 't']
on_antithyroid_medication: ['f' 't']
sick: ['f' 't']
pregnant: ['f' 't']
thyroid_surgery: ['f' 't']
I131_treatment: ['f' 't']
query_hypothyroid: ['f' 't']
query_hyperthyroid: ['f' 't']
lithium: ['f' 't']
goitre: ['f' 't']
tumor: ['f' 't']
hypopituitary: ['f' 't']
psych: ['f' 't']
TSH_measured: ['t' 'f']
T3_measured: ['t' 'f']
TT4_measured: ['t' 'f']
T4U_measured: ['t' 'f']
FTI_measured: ['t' 'f']
TBG_measured: ['f']
referral_source: ['SVHC' 'other' 'SVI' 'SVHD' 'STMW']
Class: ['negative' 'compensated_hypothyroid' 'primary_hypothyroid'
 'secondary_hypothyroid']


In [16]:
# column TGB_measured has only one unique value, so we can drop it
df.drop('TBG_measured', axis=1, inplace=True)

# Encoding Categorical Columns

To convert categorical columns into numerical ones, various techniques such as one-hot encoding for nominal data and label encoding for ordinal data are used.

## Nominal Data
For nominal data (where the categories do not have an inherent order), we can use one-hot encoding.

### Example
- **Column Name**: `color`
- **Categories**: `['Red', 'Green', 'Blue']`

**Original Data**:
| color |
|-------|
| Red   |
| Green |
| Blue  |
| Red   |
| Blue  |

**One-Hot Encoding**:
| color_Red | color_Green | color_Blue |
|-----------|-------------|------------|
| 1         | 0           | 0          |
| 0         | 1           | 0          |
| 0         | 0           | 1          |
| 1         | 0           | 0          |
| 0         | 0           | 1          |

## Ordinal Data
For ordinal data (where the categories have a specific order), we can use label encoding.

### Example
- **Column Name**: `education_level`
- **Categories**: `['High School', 'Bachelor', 'Master', 'PhD']`

**Original Data**:
| education_level |
|-----------------|
| Bachelor        |
| High School     |
| PhD             |
| Master          |
| Bachelor        |

**Label Encoding**:
| education_level |
|-----------------|
| 1               |  # Bachelor
| 0               |  # High School
| 3               |  # PhD
| 2               |  # Master
| 1               |  # Bachelor


In [17]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Label Encoding for binary ordinal columns
binary_cols = ['sex', 'on_thyroxine', 'query_on_thyroxine', 'on_antithyroid_medication', 'sick', 
               'pregnant', 'thyroid_surgery', 'I131_treatment', 'query_hypothyroid', 
               'query_hyperthyroid', 'lithium', 'goitre', 'tumor', 'hypopituitary', 'psych',
               'TSH_measured', 'T3_measured', 'TT4_measured', 'T4U_measured', 'FTI_measured']

label_encoder = LabelEncoder()
for col in binary_cols:
    df[col] = label_encoder.fit_transform(df[col])

# One-Hot Encoding for 'referral_source'
df = pd.get_dummies(df, columns=['referral_source'], drop_first=True,)



In [18]:
df["Class"].value_counts()

Class
negative                   3341
compensated_hypothyroid     187
primary_hypothyroid          91
secondary_hypothyroid         2
Name: count, dtype: int64

In [19]:
# Since "secondary_hypothyroid" category has only 2 instances, dropping it as it adds no value to the model,
# and including it could unnecessarily increase model complexity and noise.
# Dropping the column to streamline the dataset.

# Drop rows where 'Class' is 'secondary_hypothyroid'
df = df[df['Class'] != 'secondary_hypothyroid']


In [20]:
# Target Encoding for 'Class'
class_encoder = LabelEncoder()
df['Class'] = class_encoder.fit_transform(df['Class'])

In [21]:
df["Class"].value_counts()

Class
1    3341
0     187
2      91
Name: count, dtype: int64

In [22]:
#importing RandomOverSampler to manage the imbalance
from imblearn.over_sampling import SMOTENC,RandomOverSampler
x = df.drop(['Class'],axis=1)
y = df['Class']
r = RandomOverSampler()
x_sampled,y_sampled  = r.fit_resample(x,y)

## Model Training

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics  import roc_auc_score,accuracy_score

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
X_train, X_test, y_train, y_test = train_test_split(x_sampled, y_sampled, test_size=0.3, random_state=2)

In [25]:
rfc_model = RandomForestClassifier(n_estimators=1000)

In [26]:
rfc_model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000)

In [27]:
rfc_pred = rfc_model.predict(X_test)

In [28]:
#checking evaluation metrics
print(confusion_matrix(y_test, rfc_pred))

[[850   7 140]
 [145 759 109]
 [ 22   0 975]]


In [29]:
print(classification_report(y_test, rfc_pred))

              precision    recall  f1-score   support

           0       0.84      0.85      0.84       997
           1       0.99      0.75      0.85      1013
           2       0.80      0.98      0.88       997

    accuracy                           0.86      3007
   macro avg       0.87      0.86      0.86      3007
weighted avg       0.88      0.86      0.86      3007



###  Hyperparameter tuning

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

def get_best_params_for_random_forest(train_x, train_y):
    """
    Method Name: get_best_params_for_random_forest
    Description: Get the parameters for Random Forest Algorithm which give the best accuracy.
                 Use Hyperparameter Tuning.
    Output: The model with the best parameters
    On Failure: Raise an Exception
    """
    print('Entered the get_best_params_for_random_forest method')

    try:
        # Initialize with different combinations of parameters
        param_grid = {
            "n_estimators": [50, 100, 150, 200],  # Adjusting number of trees in the forest
            "criterion": ['gini', 'entropy'],
            "max_depth": [None, 10, 20, 30],  # Adjusting maximum depth of each tree
            "max_features": ['auto', 'sqrt', 'log2']  # Adjusting the number of features to consider at every split
        }

        clf = RandomForestClassifier()

        # Create an object of the Grid Search class
        grid = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, verbose=3)

        # Find the best parameters
        grid.fit(train_x, train_y)

        # Extract the best parameters
        best_params = grid.best_params_
        criterion = best_params['criterion']
        max_depth = best_params['max_depth']
        max_features = best_params['max_features']
        n_estimators = best_params['n_estimators']

        # Create a new model with the best parameters
        clf = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion,
                                     max_depth=max_depth, max_features=max_features)

        # Train the new model
        clf.fit(train_x, train_y)

        print('Random Forest best params:', best_params)
        print('Exited the get_best_params_for_random_forest method')

        return clf

    except Exception as e:
        raise Exception("Error occurred during parameter tuning")


In [31]:
random_forest_classifier = get_best_params_for_random_forest(X_train, y_train)

Entered the get_best_params_for_random_forest method
Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END criterion=gini, max_depth=None, max_features=auto, n_estimators=50;, score=0.855 total time=   0.0s


c:\Users\priks\anaconda3\envs\side\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\priks\anaconda3\envs\side\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 2/5] END criterion=gini, max_depth=None, max_features=auto, n_estimators=50;, score=0.852 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=None, max_features=auto, n_estimators=50;, score=0.860 total time=   0.0s


c:\Users\priks\anaconda3\envs\side\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\priks\anaconda3\envs\side\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 4/5] END criterion=gini, max_depth=None, max_features=auto, n_estimators=50;, score=0.882 total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=None, max_features=auto, n_estimators=50;, score=0.855 total time=   0.0s


...

[CV 4/5] END criterion=entropy, max_depth=30, max_features=auto, n_estimators=200;, score=0.885 total time=   0.4s


c:\Users\priks\anaconda3\envs\side\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 5/5] END criterion=entropy, max_depth=30, max_features=auto, n_estimators=200;, score=0.855 total time=   0.4s
[CV 1/5] END criterion=entropy, max_depth=30, max_features=sqrt, n_estimators=50;, score=0.851 total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=30, max_features=sqrt, n_estimators=50;, score=0.853 total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=30, max_features=sqrt, n_estimators=50;, score=0.862 total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=30, max_features=sqrt, n_estimators=50;, score=0.886 total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=30, max_features=sqrt, n_estimators=50;, score=0.854 total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=30, max_features=sqrt, n_estimators=100;, score=0.852 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=30, max_features=sqrt, n_estimators=100;, score=0.853 total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=30, max_features=sqrt, n_estimators=100;, s

c:\Users\priks\anaconda3\envs\side\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\priks\anaconda3\envs\side\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Random Forest best params: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'auto', 'n_estimators': 150}
Exited the get_best_params_for_random_forest method


In [32]:
#checking for accuracy parameters
from sklearn.metrics import accuracy_score

random_forest_classifier.fit(X_train, y_train)

# Make predictions on the test set
predictions = random_forest_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)

# Print the accuracy
print("Accuracy:", accuracy)

c:\Users\priks\anaconda3\envs\side\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Accuracy: 0.8623212504156967


The purpose of the given code snippet is to utilize Random Forest for hyperparameter tuning. Hyperparameter tuning aims to find the best combination of hyperparameters for the Random Forest classifier, resulting in the highest accuracy.

Random Forest is a popular choice for hyperparameter tuning due to several advantages it offers:

Ensemble Learning: Random Forest is an ensemble learning algorithm that combines multiple decision trees. By aggregating the predictions of these trees, Random Forest can provide more accurate and robust predictions compared to a single decision tree.

Flexibility: Random Forest provides flexibility in selecting hyperparameters, such as the number of trees (n_estimators), splitting criterion (criterion), maximum tree depth (max_depth), and maximum number of features to consider (max_features). This flexibility allows for fine-tuning the model to discover the best combination of hyperparameters.

Overfitting Reduction: Random Forest is less susceptible to overfitting than individual decision trees. It achieves this by randomly selecting subsets of features and data during training, resulting in a more generalized model that performs well on unseen data.

Feature Importance: Random Forest provides a measure of feature importance, which indicates the relevance of each feature in making predictions. This information can be valuable for feature selection and gaining insights into the underlying patterns in the dataset.

By employing Random Forest for hyperparameter tuning, the get_best_params_for_random_forest function aims to identify the optimal combination of hyperparameters that maximizes the classifier's accuracy on the given dataset
